In [1]:
import sys
import math
import string
import operator
import random
import collections
import datetime
import itertools
import functools

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

import IPython
import sympy as sp
import sympy.plotting as splt
import sympy.physics.vector as spv
import sympy.physics.mechanics as spm
import scipy.constants as spc

import IPython.display as ipd
spv.init_vprinting()
%matplotlib inline

In [2]:
print( f"""
    Python version {sys.version}
    IPython version {IPython.__version__}
    Numpy version {np.__version__}
    mathplotlib version {matplotlib.__version__}
    Pandas version {pd.__version__}
    Seaborn version {sns.__version__}
    """
)


    Python version 3.9.13 (main, Oct 13 2022, 21:23:06) [MSC v.1916 64 bit (AMD64)]
    IPython version 8.4.0
    Numpy version 1.23.3
    mathplotlib version 3.5.2
    Pandas version 1.4.4
    Seaborn version 0.12.0
    


In [3]:
def directory(obj):
    return [ 
        str for str in dir(obj) 
#        if callable(getattr(obj, str)) & ~str.startswith('_')
        if not str.startswith('_')
    ]

In [4]:

HALF = sp.S.Half
PI = sp.pi
E = sp.exp
POSITIVEINFINITY = sp.S.Infinity

In [5]:
def reference_frame(
    frame: str,
    x=r'\imath', y=r'\jmath', z=r'\mathbf k'
) -> spv.ReferenceFrame:
    return spv.ReferenceFrame(
        frame, latexs=(
            fr'\; {{}}^\mathcal {frame} \hat {x}',
            fr'\;{{}}^\mathcal {frame} \hat {y}',
            fr'\: {{}}^\mathcal {frame} \hat {{z}}'
        )
    )


def vector(F: spv.ReferenceFrame, rx, ry, rz=0) -> spv.Vector:
    return rx*F.x + ry*F.y + rz*F.z


def vector_cos(
    F: spv.ReferenceFrame, magnitude,
    anglex, angley, anglez=sp.pi/2
) -> spv.Vector:
    return (magnitude *
            (sp.cos(anglex)*F.x
             + sp.cos(angley)*F.y
             + sp.cos(anglez)*F.z
             )
            )


def vector_line(
    start: spv.Vector, finish: spv.Vector
) -> spv.Vector:
    return finish - start


def vector_line_eqn(
    F: spv.ReferenceFrame, start: spv.Vector, finish: spv.Vector, kappa
) -> spv.Vector:
    return start + vector_line(start, finish).normalize()*kappa


def angle_between_vectors(a: spv.Vector, b: spv.Vector):
    return sp.acos(a.dot(b)/a.magnitude()/b.magnitude())


__Solutions to equilibrium equations__

In [6]:
def solve_equilibrium_equation(
    frame: spv.ReferenceFrame,
    unknown_variables: list[sp.Symbol],
    forces: list[spv.Vector],
    moments: list[spv.Vector]
):
    """Solve a set of vectors for unknowns

    Args:
        frame (spv.ReferenceFrame): Reference frame containing vectors
        unknown_variables (list[sp.Symbol]): List of unknwn variabes in 
        the vectors which are to be solved for.

        forces (list[spv.Vector]): List of force vectors that are in 
        equilibrium

        moments (list[spv.Vector]): List of moment vectors that are in 
        equilibrium

    Returns:
        _type_: _description_
    """
    total_force = sum(forces)
    total_moments = sum(moments)
    # display(total_force)
    # display(total_moments)
    eqn = sp.Eq(
        sp.Matrix.vstack(
            total_force.to_matrix(frame),
            total_moments.to_matrix(frame)),
        sp.zeros(6, 1)
    )
    # display(eqn)
    return sp.solve(eqn, unknown_variables, dict=True)


[Distributed load calculation](https://engineeringstatics.org/distributed-loads.html)

In [7]:
def distributed_load(
    load_distribution_expr: sp.core.expr.Expr,
    integration_variable: sp.core.expr.Expr,
    upper: sp.core.expr.Expr,
    lower: sp.core.expr.Expr = 0,
) -> sp.core.expr.Expr:

    equivalent_force = sp.integrate(
        load_distribution_expr,
        [integration_variable, lower, upper]
    ).simplify()

    moment = sp.integrate(
        integration_variable*load_distribution_expr,
        [integration_variable, lower, upper]
    ).simplify()

    position_of_equivalent_force = sp.symbols(r"\overline{x}")
    eqn = sp.Eq(equivalent_force*position_of_equivalent_force, moment)
    point_of_application = sp.solve(eqn, position_of_equivalent_force)
    return equivalent_force, point_of_application[0].simplify()


x = sp.symbols("x")
assert distributed_load(sp.Rational(10, 6)*x, x, 6) == (30, 4)
assert distributed_load(4, x, 4) == (16, 2)
